In [1]:
import pygame, os, math, random
import numpy as np

colours = {"cyan": (0, 255, 255, 255),
           "blue": (0, 0, 255, 255),
           "orange": (255, 165, 0),
           "yellow": (255, 255, 0, 255),
           "green": (0, 255, 0, 255),
           "purple": (160, 32, 240, 255),
           "red": (255, 0, 0),
           "white": (250, 250, 250),
           "grey": (140, 140, 140)
           }

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [53]:
os.environ['SDL_VIDEO_CENTERED'] = '1'
pygame.init()
width, height = 1000, 700
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Perspective projection")
clock = pygame.time.Clock()

# distance from camera point to screen
d = 200
# camera point
c = np.array([0.0, 200.0, 0.0])
# angles of rotation
theta = 0
phi = 0
psi = 0

def rotate_point(p, theta, phi, psi):
    """
    Input: point, angles of rotation about x, y, z axis
    Output: point p': p rotated about origin by angles
    """
    A = np.array([[1, 0, 0],
                  [0, np.cos(theta), np.sin(theta)],
                  [0, -np.sin(theta), np.cos(theta)]])
    B = np.array([[np.cos(phi), 0, -np.sin(phi)],
                  [0, 1, 0],
                  [np.sin(phi), 0, np.cos(phi)]])
    C = np.array([[np.cos(psi), np.sin(psi), 0],
                  [-np.sin(psi), np.cos(psi), 0],
                  [0, 0, 1]])
    
    # rotate about y axis first: bring objects in front of screen before rotating about x axis
    return C @ A @ B @ p

def cart_to_pyg(cart):
    """
    Input: (x, y, z) coordinates
    Output: (x', y') coordinates to display on screen centered at (0,0)

    (drops z = d coordinate)

    Pygame has origin top-left, with x increasing right, y down so transform
    final coordinates to display
    """
    return (width/2 + cart[0], height/2 - cart[1])

def project_point(p, type="P"):
    """
    Input: (x, y, z) coordinate of point, type of projection: parallel, orthogonal
    Output: (x', y', d) coordinates of projection on view plane z = d
    """
    # find relative distance to camera
    p_prime = p - c
    # rotate 
    p_prime = rotate_point(p_prime, theta, phi, psi)
    # if closer than view distance (or behind camera) denote
    if p_prime[2] < d:
        vis = False
    else:
        vis = True
    # project onto screen
    if type == "P":
        p_prime =  p_prime * d/p_prime[2]
    elif type =="O":
        p_prime[2] = d
    return {'pos': p_prime, 'vis': vis}

def draw_line(p1, p2, colour):
    """
    Input: 3D endpoints of line, colour
    Output: 2D line drawn via screen projection
    """
    # project both endpoints
    s1 = project_point(p1)
    s2 = project_point(p2)
    # if both endpoints visible: draw
    if s1['vis'] and s2['vis']:
        pygame.draw.line(window, colour, cart_to_pyg(s1['pos']), cart_to_pyg(s2['pos']))

def draw_rectangle(p, width, height, depth, colour):
    """
    Input: 3D point of bottom left, width, height, depth values, colour
    Output: draw wireframe cube onto surface
    """
    x = p[0]
    y = p[1]
    z = p[2]
    lines = [
        # bottom face
        [(x, y, z), (x, y, z + depth)],
        [(x, y, z + depth), (x + width, y, z + depth)],
        [(x + width, y, z + depth), (x + width, y, z)],
        [(x + width, y, z), (x, y, z)],
        # top
        [(x, y + height, z), (x, y + height, z + depth)],
        [(x, y + height, z + depth), (x + width, y + height, z + depth)],
        [(x + width, y + height, z + depth), (x + width, y + height, z)],
        [(x + width, y + height, z), (x, y + height, z)],
        # verticals
        [(x, y, z), (x, y + height, z)],
        [(x, y, z + depth), (x, y + height, z + depth)],
        [(x + width, y, z + depth), (x + width, y + height, z + depth)],
        [(x + width, y, z), (x + width, y + height, z)]
    ]
    for points in lines:
        draw_line(points[0], points[1], colour)

def draw_polygon(points, colour):
    """
    Input: list of 3D vertices, colour
    Output: draw polygon to screen
    """
    # project each point
    visible = []
    projected = []
    for p in points:
        s = project_point(np.array(p))
        visible.append(s['vis'])
        projected.append(cart_to_pyg(s['pos']))
    # draw if all are visible
    if all(visible):
        pygame.draw.polygon(window, colour, projected)

def input(c, theta, phi, psi):
    """
    Accept user input for movment and rotation of viewing plane and basis
    """
    # direction to move in
    move = np.array([0.0, 0.0, 0.0])
    # key presses
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()
        elif event.type == pygame.MOUSEMOTION:
            # relative moouse movement
            x = event.rel[0]
            y = event.rel[1]
            # left, right mouse movement
            if x < -5:
                phi -= 0.1
            elif x > 5:
                phi += 0.1
            # up, down mouse movement
            if y < -5:
                # prevent looking up past vertical
                if theta > -math.pi/2:
                    theta -= 0.1
            elif y > 5:
                # prevent looking down past vertical
                if theta < math.pi/2:
                    theta += 0.1
            # re-centre mouse
            pygame.mouse.set_pos(width/2, height/2)
    # keys held
    keys = pygame.key.get_pressed()
    # movement forward, backward, left, right
    if keys[pygame.K_w]:
        move[2] += 5
    if keys[pygame.K_s]:
        move[2] -= 5
    if keys[pygame.K_a]:
        move[0] -= 5
    if keys[pygame.K_d]:
        move[0] += 5
    # rotate about y-axis: left, right
    if keys[pygame.K_LEFT]:
        phi -= 0.1
    if keys[pygame.K_RIGHT]:
        phi += 0.1
    # rotate about x-axis: up, down
    if keys[pygame.K_UP]:
        # prevent looking up past vertical
        if theta > -math.pi/2:
            theta -= 0.1
    if keys[pygame.K_DOWN]:
        # prevent looking down past vertical
        if theta < math.pi/2:
            theta += 0.1
    """
    # rotate about z-axis: (,),(.)
    if keys[pygame.K_COMMA]:
        psi += 0.1
    if keys[pygame.K_PERIOD]:
        psi -=  0.1
    # move along x axis: a, d
    if keys[pygame.K_a]:
        a[0] -= 5
    if keys[pygame.K_d]:
        a[0] += 5
    # move along y axis: space, shift
    if keys[pygame.K_SPACE]:
        a[1] += 5
    if keys[pygame.K_LSHIFT]:
        a[1] -= 5
    # move along z axis: w, s
    if keys[pygame.K_w]:
        a[2] += 5
    if keys[pygame.K_s]:
        a[2] -= 5
    """
    # rotate movement direction in opposite rotation to points (reverse transform)
    # set an angle to zero to not move in that direction
    # e.g. keeping theta angle as 0 stops moving in y axis when looking up/down
    move = rotate_point(move, 0, -phi, -psi)
    # move camera position by direction
    c += move

    return c, theta, phi, psi

def get_colours(n):
    """
    Input: number
    Output: list of n random colour values as RGB tuple
    """
    col_list = []
    for i in range(n):
        col_list.append(random.choice(list(colours.values())))
    return col_list

def cube_grid(m, n):
    """
    Add grid of m x n cubes to polygon list with random coloured faces
    """
    # generate colours
    cols = get_colours(25)
    # colour index
    k = 0
    # grid of columns: slight offset to prevent zero errors
    for i in range(-m//2, m//2):
        for j in range(-n//2, n//2):
            pos = np.array([j, 0, i]) * 200 + np.array([1, 0, 1])
            x_off = np.array([100, 0, 0])
            y_off = np.array([0, 100, 0])
            z_off = np.array([0, 0, 100])
            # side faces
            polygons.append([pos, pos+x_off, pos+x_off+y_off,pos+y_off,cols[k]])
            polygons.append([pos, pos+z_off, pos+z_off+y_off,pos+y_off,cols[k+1]])
            polygons.append([pos+x_off, pos+x_off+z_off, pos+x_off+z_off+y_off,pos+x_off+y_off,cols[k+2]])
            polygons.append([pos+z_off, pos+x_off+z_off, pos+x_off+z_off+y_off,pos+z_off+y_off,cols[k+3]])
            # bottom face
            polygons.append([pos, pos+x_off, pos+x_off+z_off, pos+z_off, cols[k+4]])
            # top face
            polygons.append([pos+y_off, pos+x_off+y_off, pos+x_off+y_off+z_off, pos+y_off+z_off, cols[k+5]])
            k = (k + 1) % 20

def centroid_dist(poly):
    """
    Input: list points and colour of polygon
    Output: distance between centroid of points and camera
    """
    points = np.array(poly[:-1])
    return np.linalg.norm(points - c)

# initialise polygon list
polygons = []

# add cube grid
cube_grid(4, 4)

# game loop
while True:
    # draw black background
    pygame.draw.rect(window, (0, 0, 0), pygame.Rect(0, 0, width, height))
    # accept input
    c, theta, phi, psi = input(c, theta, phi, psi)
    # set framerate
    clock.tick(60)
    print(clock.get_fps())
    """
    k = 0
    # grid of columns: slight offset to prevent zero errors
    for i in range(-4, 4):
        for j in range(-4, 4):
            pos = np.array([j, 0, i]) * 500 + np.array([1, 0, 1])
            if np.linalg.norm(pos - c) < 1000:
                draw_rectangle(pos, 100, 1000, 100, cols[k])
            k += 1
    """
    # sort polygons by centroid: decreasing distance
    polygons = sorted(polygons, key=lambda poly: centroid_dist(poly), reverse=True)

    # draw in order: farthest away first, closer on top
    for poly in polygons:
        # limit distance up to which polgons are drawn
        if centroid_dist(poly) < 2000:
            draw_polygon(poly[:-1], poly[-1])


    # display
    pygame.event.pump()
    pygame.display.update()


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
36.10108184814453
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
43.103450775146484
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
50.50505065917969
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
53.47593307495117
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
51.81346893310547
55.5555534362793
55.5555534362793
55.5555534362793

error: video system not initialized